# Fantasy Premier League 2022/2023 Season Analysis:

**This notebook covers how to access the Fantasy Premier League API, build a dataframe, and analyze  the data using Jupyter, Python, Pandas, and Matplotlib.**

**About the game**: Each FPL manager is given a starting budget of £100 million and must pick a total of 15 players: two goalkeepers, five defenders, five midfielders and three forwards. You are limited to a maximum of three players from each Premier League team. Players in your team score points based on real-life performances with the biggest contributors being goals, assists and cleansheets.

**Analysis Limits**: The data used in this notebook includes all season totals at the end of Gameweek 38 of the 2022/2023 season. This is more of a season overview analysis than a week-to-week analysis, therefore, form and fixtures were not accounted for.  I have created a 'price range' category to make the analysis more accurate as price changes generally happen within their price range.

## 1. Data Loading

First, we import the necessary libraries that we will use throughout the analysis. We then use the requests library retrieve data via the FPL API endpoint. (https://fantasy.premierleague.com/api/bootstrap-static/). The next step is to convert the response data into a json object and then again into a pandas Dataframe. 


In [1]:
import requests
import pandas as pd
import numpy as np

In [3]:
#Get data and convert to json

url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
r = requests.get(url)
fpl_json = r.json()

In [4]:
#View dict keys
fpl_json.keys()

dict_keys(['elements', 'element_types', 'element_stats', 'events', 'fixtures', 'settings', 'teams'])

In [5]:
#Convert json objects to dataframe. 
elements_df = pd.DataFrame(fpl_json['elements'])
elements_types_df = pd.DataFrame(fpl_json['element_types'])
element_stats_df = pd.DataFrame(fpl_json['element_stats'])
teams_df = pd.DataFrame(fpl_json['teams'])

#Save as csv
elements_df.to_csv('data/22_23_season_data/elements.csv')
elements_types_df.to_csv('data/22_23_season_data/element_types.csv')
element_stats_df.to_csv('data/22_23_season_data/element_stats.csv')
teams_df.to_csv('data/22_23_season_data/teams.csv')

## 2. Data Cleaning

The next step involves cleaing the data

### Understand data

In [15]:
elements_df.head()

,id,name,now_cost,position,team,clean_sheets_per_90,threat_rank_type,expected_assists_per_90,expected_assists,points_per_game_rank,...,threat_rank,ep_this,transfers_in,bps,goals_conceded_per_90,selected_by_percent,influence_rank_type,points_per_game_rank_type,clean_sheets,now_cost_rank
0,3,Granit Xhaka,48,MID,Arsenal,0.39,38,0.12,3.89,47,...,62,7.3,942505,634,1.05,2.7,23,23,13,265
1,4,Mohamed Elneny,41,MID,Arsenal,0.00,233,0.03,0.04,457,...,593,0.0,202760,27,1.62,0.6,226,201,0,634
2,5,Rob Holding,42,DEF,Arsenal,0.00,118,0.02,0.15,405,...,341,0.7,23338,120,2.08,0.2,137,144,0,598
3,6,Thomas Partey,47,MID,Arsenal,0.40,90,0.08,2.17,210,...,152,1.9,110171,468,1.02,0.4,47,80,11,311
4,7,Martin Ødegaard,69,MID,Arsenal,0.37,8,0.23,8.02,7,...,18,6.9,7014051,813,1.09,33.3,3,3,13,44


In [16]:
elements_df.shape

(778, 75)

In [17]:
elements_df.columns

Index(['id', 'name', 'now_cost', 'position', 'team', 'clean_sheets_per_90',
       'threat_rank_type', 'expected_assists_per_90', 'expected_assists',
       'points_per_game_rank', 'goals_scored', 'penalties_missed',
       'creativity_rank_type', 'transfers_out', 'value_form',
       'direct_freekicks_order', 'value_season', 'bonus', 'starts_per_90',
       'cost_change_start', 'news_added', 'expected_goals_conceded',
       'cost_change_start_fall', 'expected_goals_conceded_per_90', 'red_cards',
       'threat', 'selected_rank_type', 'influence', 'penalties_saved',
       'corners_and_indirect_freekicks_order', 'ep_next', 'event_points',
       'web_name', 'creativity', 'ict_index_rank', 'saves_per_90',
       'creativity_rank', 'expected_goals', 'own_goals', 'status',
       'now_cost_rank_type', 'saves', 'yellow_cards', 'goals_conceded', 'news',
       'expected_goal_involvements_per_90', 'assists', 'form_rank_type',
       'ict_index_rank_type', 'chance_of_playing_next_round', 'in

### Data Tansformation

Get only the interesting columns for analysis

In [23]:
cols = ['now_cost', 'position', 'team', 'clean_sheets_per_90',
       'expected_assists_per_90', 'expected_assists',
        'goals_scored', 'value_form', 'value_season', 'bonus',  'expected_goals_conceded',
        'expected_goals_conceded_per_90', 'threat',  'influence', 'penalties_saved',
       'web_name', 'creativity','expected_goals','saves', 'goals_conceded',
       'expected_goal_involvements_per_90', 'assists', 'ict_index', 'form', 
        'dreamteam_count','expected_goal_involvements', 'starts',
       'points_per_game', 'minutes', 'total_points',  'expected_goals_per_90', 'bps', 'goals_conceded_per_90',
       'selected_by_percent','clean_sheets']

df = elements_df[cols]

Check data type

In [24]:
df.dtypes

now_cost                               int64
position                              object
team                                  object
clean_sheets_per_90                  float64
expected_assists_per_90              float64
expected_assists                     float64
goals_scored                           int64
value_form                           float64
value_season                         float64
bonus                                  int64
expected_goals_conceded              float64
expected_goals_conceded_per_90       float64
threat                               float64
influence                            float64
penalties_saved                        int64
web_name                              object
creativity                           float64
expected_goals                       float64
saves                                  int64
goals_conceded                         int64
expected_goal_involvements_per_90    float64
assists                                int64
ict_index 

In [27]:
#Map team to team name
team_codes =list(df['team'].unique())
team_names = list(teams_df['short_name'].unique())
teams_dict = dict(zip(team_codes, team_names))

df.loc[:, 'team'] = df.loc[:,'team'].apply(lambda x: teams_dict[x])

#Create cost column
df.loc[:, 'price'] = df.loc[:,'now_cost'] / 10

#Create G+A column

df.loc[:, 'G+A'] = df.loc[:,'goals_scored'] + df.loc[:,'assists']

C:\Users\Luto Sibotoboto\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)
C:\Users\Luto Sibotoboto\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)


Drop uninteresting columns

In [29]:
df = df.drop(['now_cost'], axis=1)

Check for duplicated rows

In [30]:
df.duplicated().sum()

0

Discretization and Binning

In [31]:
df['position'].value_counts().index

Index(['MID', 'DEF', 'FWD', 'GKP'], dtype='object')

In [40]:
#Create labels
labels = ['Budget', 'Mid', 'Premium']

#Create bins
gk_bins = [3.0, 5.0, 6.0, 7.0]
def_bins = [3.0, 5.0, 6.0, 8.0]
mid_bins = [3.0, 6.0, 9.0, 14.0]
for_bins = [3.0, 6.0, 9.0, 14.0]

#Create list of tuples
tup = [(df, gk_bins, 'GKP', labels),
       (df, def_bins, 'DEF', labels),
       (df, mid_bins, 'MID', labels),
       (df, for_bins, 'FWD', labels)
       ]

#Discretinization function
def categorize_price(dataframe, bins, position, labels):
    grp = dataframe.groupby(['position']).get_group(position)
    grp_prices = grp['price']
    grp.loc[:,'price_range'] = pd.cut(grp_prices, bins, labels=labels, right=False)
    
    grp_lst.append(grp)
    return grp

#Create grp_list
grp_lst = list()

for item in tup:
    df = item[0]
    bins = item[1]
    pos = item[2]
    labels = item[3]
    categorize_price(df, bins, pos, labels)
    
frames = grp_lst
df = pd.concat(frames)


C:\Users\Luto Sibotoboto\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [41]:
#Create labels
labels = ['Differential', 'High']

#Create bins
selected_by = [0.0, 15.0, 99.0]

#Create list of tuples
tup = (df, selected_by, labels)

#Discretinization function
def categorize_own(dataframe, bins, labels):
    ownership = dataframe['selected_by_percent']
    dataframe['ownership'] = pd.cut(ownership, bins, labels=labels, right=False)

    return dataframe

df = categorize_own(df, selected_by, labels)

### Handle missing data

In [43]:
df[df.isnull().any(axis=1)]

,position,team,clean_sheets_per_90,expected_assists_per_90,expected_assists,goals_scored,value_form,value_season,bonus,expected_goals_conceded,...,total_points,expected_goals_per_90,bps,goals_conceded_per_90,selected_by_percent,clean_sheets,price,G+A,price_range,ownership


### Save clean data

In [44]:
df.to_csv('data/22_23_season_data/fpl_clean.csv', index=False)